In [5]:
import gc
import numpy as np

"""STVM 계산(3차로 점선) + 지상부 추가"""
import pandas as pd

import os


# FIX 값 모음
###################################################################################################################
start_interval = 1800
end_interval = 5400

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1
}

vehicle_types = [100, 300, 630, 640, 650]
######## 검지기 #############
# 지상부_진입 검지기(결과 데이터는 지상부_진입의 6번째 데이터 부터 들어가므로 해당 검지기를 선정함)
enter_line = 6
# 본선부 검지기
main_line = 60
############################


######## 램프 ###############
# 유입램프
input_ramp = 59

# 유출램프
output_ramp = 61
############################

######## 구간 ###############
# 진입부_지상
entry_point = [i for i in range(6,11)]

# 본선부
middle_point = [i for i in range(41,241)]

# 진출부_ 지상
exit_point = [i for i in range(271,276)]
############################

###################################################################################################################

# 함수 모음
###################################################################################################################

# 평균속도
def speed_mean(df):
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        df.groupby(["TimeGroup", "New_Measurement"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["V_next"] = speed_mean_df.groupby("TimeGroup")["V_mean"].shift(-1)
    speed_mean_df["delta_V"] = (speed_mean_df["V_next"] - speed_mean_df["V_mean"]) / speed_mean_df["V_mean"]
    speed_mean_df["delta_V"] = speed_mean_df["delta_V"].fillna(0)
    return speed_mean_df

# 밀도
def density_mean(df):
    density_mean_df = df.assign(K = df["V_count"] * 12 / df["V_mean"])
    density_mean_df["K_next"] = density_mean_df.groupby("TimeGroup")["K"].shift(-1)
    density_mean_df["delta_K"] = (density_mean_df["K_next"] - density_mean_df["K"]) / density_mean_df["K"]
    density_mean_df["delta_K"] = density_mean_df["delta_K"].fillna(0)
    return density_mean_df

# 가중 중차량 비율
def heavy_rate(original_df):

    # 진입부 검지기 선정
    measurement = enter_line

    # TimeGroup별 중차량 갯수 집계
    heavy_df = (
        original_df[
            (original_df["New_Measurement"] == measurement) &
            (original_df["Vehicle type"].isin([630,640,650]))
        ]
        .groupby("TimeGroup")
        .size()
        .reset_index(name="heavy_count")
    )

    # TimeGroup별 총 차량 갯수 집계
    total_df = (
        original_df[original_df["New_Measurement"] == measurement]
        .groupby("TimeGroup")
        .size()
        .reset_index(name="total_count")
    )

    heavy_rate_df = pd.merge(heavy_df, total_df, on="TimeGroup", how="left")

    heavy_rate_df["rate"] = heavy_rate_df["heavy_count"] / heavy_rate_df["total_count"]
    return heavy_rate_df


# 진입 포화도
def entry_saturation(original_df):
    # 실측용량 C
    max_capacity = 2200
    entry_saturation_df = (
        # 진입부에서 유입된 교통량이므로 진입부 중 한 개의 검지기를 선정하여 측정
        original_df[original_df["New_Measurement"] == enter_line]
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="entry_volume")  # 차량 수를 entry_volume이라는 컬럼명으로
    )

    # 단위가 대/시 이기 때문에 현재 5분집계 * 12
    entry_saturation_df["Phi_진입"] = entry_saturation_df["entry_volume"] * 12 / max_capacity

    return entry_saturation_df

# 램프 유출입 비율
def rfr_rate(original_df):
    """
    1. 본선 1개의 검지기 data 수집(New_Measurement = 60)
    2. 위의 검지기 앞, 뒤 검지기 data (59 / 61)
    3. RFR 연산
    """
    main_df = (
        original_df[original_df["New_Measurement"] == main_line] # 60
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="main_line")
    )
    input_df = (
        original_df[original_df["New_Measurement"] == input_ramp] # 59
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="input_ramp")
    )
    output_df = (
        original_df[original_df["New_Measurement"] == output_ramp] # 61
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="output_ramp")
    )
    #main_df["RFR"] = (input_df["input_ramp"] + output_df["output_ramp"])/main_df["main_line"]
    # 유출램프 구현이 안 됐기 때문에 0.2값으로 fix
    main_df["RFR"] = 0.2
    return main_df

# 유출 정상성 비율
def output_normality(original_df):
    # 진입부
    entry_df = (
        original_df[original_df["New_Measurement"].isin(entry_point)]
        .groupby(["TimeGroup"])
        .size()
        .reset_index(name="entry_volume")
    )

    # 진출부
    exit_df = (
        original_df[original_df["New_Measurement"].isin(exit_point)]
        .groupby(["TimeGroup"])
        .size()
        .reset_index(name="exit_volume")
    )
    normality_df = pd.concat([entry_df, exit_df["exit_volume"]], axis=1)
    normality_df["F(outrate)"] = normality_df["exit_volume"] / normality_df["entry_volume"]
    normality_df["1-F(outrate)"] = 1 - normality_df["F(outrate)"]

    return normality_df

def calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df):

    # TimeGroup 기준으로  Merge
    merged = (
    speed_df[["TimeGroup", "New_Measurement", "delta_V"]]
    .merge(density_df[["TimeGroup", "New_Measurement", "delta_K"]], on=["TimeGroup", "New_Measurement"])
    .merge(heavy_df[["TimeGroup", "rate"]], on="TimeGroup")
    .merge(entry_saturation_df[["TimeGroup", "Phi_진입"]], on="TimeGroup")
    .merge(rfr_df[["TimeGroup", "RFR"]], on="TimeGroup")
    .merge(normality_df[["TimeGroup", "1-F(outrate)"]], on="TimeGroup")
    )


    merged["STVM"] = (
        weights["w1"] * merged["delta_V"] +
        weights["w2"] * merged["delta_K"] +
        weights["w3"] * merged["rate"] +
        weights["w4"] * merged["Phi_진입"] +
        weights["w5"] * merged["RFR"] +
        weights["w6"] * merged["1-F(outrate)"]
    )

    return merged[["TimeGroup", "New_Measurement", "STVM"]]


def calculate_z_score(stvm_df):
    # 평균
    mean_stvm = stvm_df["STVM"].mean(skipna=True)

    # 표준편차
    std_stvm = stvm_df["STVM"].std(skipna=True)

    # Z-Score 계산
    stvm_df["Z-Score"] = (stvm_df["STVM"] - mean_stvm) / std_stvm
    z_max = stvm_df["Z-Score"].max()
    z_min = stvm_df["Z-Score"].min()

    stvm_df["환산점수"] = stvm_df["Z-Score"].apply(lambda z : z_to_score(z, z_min, z_max))

    stvm_df = pd.pivot(stvm_df, index="TimeGroup", columns= "New_Measurement", values="환산점수")
    return stvm_df

def modify_frame(z_score_df):
    result_df = z_score_df.copy()
    result_df = result_df.drop(columns=list(range(1,6)) + list(range(276,281)))

    columns = result_df.columns.tolist()

    new_columns = [-5, -4, -3, -2, -1]
    # 나머지 (11번 이후) → 1부터 순차 번호
    new_columns += list(range(1, len(columns) - 5 + 1))
    result_df.columns = new_columns
    return result_df



def z_to_score(z, z_min, z_max):
    if 1.645 <= z <= z_max:
        return 50 + ((95 + 5 * ((z - 1.645) / (z_max - 1.645))) * 0.5)
    elif 1.282 <= z < 1.645:
        return 50 + ((90 + 5 * ((z - 1.282) / (1.645 - 1.282))) * 0.5)
    elif 1.038 <= z < 1.282:
        return 50 + ((85 + 5 * ((z - 1.038) / (1.282 - 1.038))) * 0.5)
    elif 0.842 <= z < 1.038:
        return 50 + ((80 + 5 * ((z - 0.842) / (1.038 - 0.842))) * 0.5)
    elif 0.676 <= z < 0.842:
        return 50 + ((75 + 5 * ((z - 0.676) / (0.842 - 0.676))) * 0.5)
    elif 0.526 <= z < 0.676:
        return 50 + ((70 + 5 * ((z - 0.526) / (0.676 - 0.526))) * 0.5)
    elif 0.387 <= z < 0.526:
        return 50 + ((65 + 5 * ((z - 0.387) / (0.526 - 0.387))) * 0.5)
    elif 0.255 <= z < 0.387:
        return 50 + ((60 + 5 * ((z - 0.255) / (0.387 - 0.255))) * 0.5)
    elif -0.255 <= z < 0.255:
        return 50 + ((40 + 5 * ((z + 0.255) / (0.255 + 0.255))) * 0.5)
    elif -0.387 <= z < -0.255:
        return 50 + ((35 + 5 * ((z + 0.387) / (-0.255 + 0.387))) * 0.5)
    elif -0.526 <= z < -0.387:
        return 50 + ((30 + 5 * ((z + 0.526) / (-0.387 + 0.526))) * 0.5)
    elif -0.676 <= z < -0.526:
        return 50 + ((25 + 5 * ((z + 0.676) / (-0.676 + 0.842))) * 0.5)
    elif -0.842 <= z < -0.676:
        return 50 + ((20 + 5 * ((z + 0.842) / (-0.676 + 0.842))) * 0.5)
    elif -1.038 <= z < -0.842:
        return 50 + ((15 + 5 * ((z + 1.038) / (-0.842 + 1.038))) * 0.5)
    elif -1.282 <= z < -1.038:
        return 50 + ((10 + 5 * ((z + 1.282) / (-1.038 + 1.282))) * 0.5)
    elif -1.645 <= z < -1.282:
        return 50 + ((5 + 5 * ((z + 1.645) / (-1.282 + 1.645))) * 0.5)
    elif z_min <= z < -1.645:
        return 50 + ((0 + 5 * ((z - z_min) / (-1.645 + abs(z_min)))) * 0.5)
    else:
        return np.nan

###################################################################################################################

folder_path = r"E:\network_version2.9.0\network09_mer"
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()
for i in range(len(mer_list)):
    mer_file = mer_list[i]
    with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
            # 데이터가 시작하는 인덱스 찾기
            data_start_idx = None

            for j, line in enumerate(lines):
                if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                    data_start_idx = j
                    break

            # 데이터프레임 생성
            if data_start_idx is not None:

                # 컬럼명 추출 및 공백 제거
                columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

                # 데이터 부분 추출 및 가공
                data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
                data = [line.strip().split(";") for line in data_lines if line.strip()]

                # 데이터프레임 생성
                df = pd.DataFrame(data, columns=columns)

                # 컬럼 내부 데이터 정수형 변환
                df = df.apply(pd.to_numeric, errors="coerce")

                original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)


                #불필요 컬럼 제거
                original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

                original_df["New_Measurement"] = original_df["Measurem."] % 1000

                bins = np.arange(start_interval, end_interval+1, 300)
                labels = [f"{start}~{start+300}" for start in bins[:-1]]  # 구간 라벨링

                # 구간 나누기 및 컬럼 추가
                original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)

                # 평균속도
                speed_df = speed_mean(original_df)

                # 밀도
                density_df = density_mean(speed_df)

                # 가중 중차량 비율
                heavy_df = heavy_rate(original_df)

                # 진입 포화도
                entry_saturation_df = entry_saturation(original_df)

                # 램프 유출입 비율
                rfr_df = rfr_rate(original_df)

                # 진출 정상성
                normality_df = output_normality(original_df)

                # STVM 계산
                stvm_df = calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df)

                # Z-Score 계산
                z_score_df = calculate_z_score(stvm_df)

                # 결과 정리
                result_df = modify_frame(z_score_df)

                excel_folder_path = os.path.join(folder_path, "STVM")
                os.makedirs(excel_folder_path, exist_ok=True)
                excel_file_name = f"STVM_{i+1}.xlsx"
                excel_file_path = os.path.join(excel_folder_path, excel_file_name)
                result_df.to_excel(excel_file_path, index=True)
                print(f"{excel_file_name} 생성 완료")

                # 메모리 정리
                del df, original_df, speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df, stvm_df, z_score_df
                gc.collect()

,-5,-4,-3,-2,-1,1,2,3,4,5,...,256,257,258,259,260,261,262,263,264,265
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,71.311475,71.170006,71.450992,71.170060,71.170163,71.170231,71.309311,71.032205,71.586944,70.762285,...,70.885183,71.314185,71.313053,70.884995,70.884319,72.043508,72.010110,71.170068,70.764534,71.170259
2100~2400,59.195976,59.995375,58.010406,57.584580,59.195823,59.195892,59.196070,59.195816,58.362986,61.045393,...,59.909493,59.195816,59.196094,59.546417,59.891061,58.852591,59.197014,59.541796,60.272788,59.534071
2400~2700,85.518774,85.990227,85.985742,87.385455,86.880098,84.122549,85.065415,85.062716,86.437408,83.609820,...,85.518737,85.518764,85.518733,86.057605,86.052947,86.049233,84.993257,84.986640,85.518734,84.983134
2700~3000,70.851446,70.850485,71.130492,70.850376,70.849512,70.990313,71.272167,70.850467,70.849440,71.556331,...,70.990166,70.853478,70.990057,71.127579,70.855350,71.127594,70.857275,71.264890,70.717865,70.714699
3000~3300,98.938731,98.842635,98.937099,99.033968,99.033848,99.033871,98.741573,99.231859,99.033930,98.544576,...,98.908312,98.907061,98.777968,99.034004,99.034586,98.386087,99.168916,98.899829,99.169232,99.436956
3300~3600,64.542502,64.528494,64.039410,63.566666,63.567218,64.037209,65.248182,64.021921,63.113894,64.479679,...,63.104832,64.496730,64.025930,62.654214,63.567487,66.337916,64.460426,63.130268,64.010106,61.371243
3600~3900,84.495164,84.495065,82.622940,84.495621,83.537196,84.981788,84.978782,84.015052,83.529762,84.983472,...,83.577788,84.496270,84.496195,84.495064,84.961744,83.573265,83.559116,84.495334,82.600997,85.437339
3900~4200,70.758085,70.888990,71.563443,70.629800,71.157472,71.156639,70.625954,71.023302,70.888258,71.158919,...,70.785377,70.054317,70.895807,71.151315,70.897954,71.151587,71.150530,71.276341,71.023240,71.148526
4200~4500,61.984445,60.599430,61.984208,62.932649,62.452638,61.521548,62.449954,62.447009,64.286451,61.540171,...,63.789065,65.876230,61.984225,62.400568,60.760478,61.984181,62.403476,61.569270,63.246039,61.572665


STVM_1.xlsx 생성 완료


,-5,-4,-3,-2,-1,1,2,3,4,5,...,256,257,258,259,260,261,262,263,264,265
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,71.238939,71.097272,71.378650,71.097326,71.097430,71.097497,71.236772,70.959279,71.514791,70.688982,...,70.813672,71.097366,71.240645,70.812270,71.246210,71.950082,71.234944,71.097714,70.960377,70.959311
2100~2400,58.894601,59.695115,57.707377,57.324047,58.894448,58.894517,58.894695,58.894441,58.060448,60.692336,...,59.605072,58.894535,58.894440,59.248930,59.591604,58.551318,59.240859,59.581898,59.234671,59.232874
2400~2700,85.382012,85.854123,85.849631,87.251298,86.745235,83.972960,84.922325,84.919408,86.301928,83.459514,...,85.382180,85.382069,85.383178,85.920868,85.925489,85.914659,84.844923,84.840431,85.917462,84.838273
2700~3000,71.057943,71.056981,71.337378,71.056871,71.056007,71.197004,71.479251,71.056963,71.055934,71.763811,...,71.197227,71.059686,71.196955,71.334256,71.067293,71.204655,70.925288,71.475984,70.784123,71.056745
3000~3300,98.838285,98.733093,98.836498,98.942536,98.942405,98.942430,98.622466,99.159157,98.942494,98.406824,...,98.804432,98.803228,98.661691,98.942526,99.086371,98.382295,99.232996,98.655725,99.233035,99.086054
3300~3600,65.576821,65.560068,64.738243,64.264840,64.265392,64.736039,66.084305,64.720729,63.811436,65.501690,...,63.801354,65.523378,64.725318,63.351991,63.352991,67.198433,65.491007,64.264866,64.711132,63.380345
3600~3900,84.919546,84.919446,83.044710,84.920003,83.960241,85.377013,85.374224,84.438763,83.952796,85.378576,...,84.002364,84.454640,84.919704,85.359032,85.792273,82.650449,84.445198,83.966231,84.436293,84.920121
3900~4200,70.740771,70.871859,71.547253,70.612308,71.140716,71.139882,70.608456,71.006359,70.871126,71.142165,...,71.261148,70.881251,71.006286,70.753016,71.010371,71.516819,70.758001,71.132976,70.880315,71.133032
4200~4500,61.356998,59.974635,61.356761,62.306526,61.825845,60.893455,61.823157,61.820208,63.662217,60.912104,...,62.217450,62.205034,61.356726,62.609225,59.791315,61.362313,62.194968,61.771049,62.179043,61.356791


STVM_2.xlsx 생성 완료


,-5,-4,-3,-2,-1,1,2,3,4,5,...,256,257,258,259,260,261,262,263,264,265
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,70.914772,70.769831,71.057713,70.769886,70.769992,70.770062,70.912555,70.628648,71.197001,70.352104,...,70.048914,71.070450,71.214826,70.914640,70.934638,71.479001,70.633336,70.769850,70.632005,70.770000
2100~2400,58.605871,59.424887,57.412608,57.062156,58.605714,58.605784,58.605967,58.605707,57.752438,60.397594,...,60.516389,58.606860,58.258601,58.960273,59.349187,57.591615,58.961985,58.606458,59.309841,58.258621
2400~2700,84.885674,85.377080,85.372486,86.806549,86.288790,83.413124,84.384432,84.381448,85.835236,82.887811,...,84.892722,84.885628,85.447672,85.442998,85.501095,85.978305,84.321481,83.733735,84.887014,84.885967
2700~3000,71.530953,71.529969,71.816847,71.529856,71.528972,71.673228,71.961999,71.529950,71.528898,72.253136,...,71.398406,71.530811,71.530421,71.816502,71.438394,71.687653,71.391280,71.965885,71.528213,71.528421
3000~3300,98.733003,98.618278,98.731054,98.846702,98.846559,98.846586,98.497625,99.082954,98.846656,98.262440,...,98.999825,98.696798,98.695651,98.846574,99.178285,98.277897,99.154310,98.391460,98.846567,99.312006
3300~3600,65.117658,65.100519,64.342119,63.857774,63.858339,64.339864,65.636872,64.324201,63.393891,65.040791,...,63.383098,64.334173,64.330920,62.448216,63.402440,66.800716,65.036635,65.568782,64.307923,62.517041
3600~3900,86.343998,86.343904,84.532188,86.344432,85.434496,86.806003,86.803149,85.888176,85.427438,86.807602,...,85.469167,86.787234,85.024118,86.803113,85.996348,85.500366,84.999790,86.344781,85.883462,85.413603
3900~4200,70.609575,70.743693,71.434697,70.478142,71.018764,71.017910,70.474201,70.881301,70.742942,71.020246,...,71.147683,70.493011,71.144540,71.012293,71.047475,70.638433,71.140089,70.626505,70.881355,71.140548
4200~4500,60.501491,59.222923,60.501249,61.472965,60.981174,60.027234,60.978425,60.975407,62.859991,60.046314,...,60.934299,61.359478,60.926569,59.711319,60.579594,61.778920,60.924113,59.377252,60.926274,60.081276


...

STVM_34.xlsx 생성 완료


,-5,-4,-3,-2,-1,1,2,3,4,5,...,256,257,258,259,260,261,262,263,264,265
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,55.209141,55.358021,55.803328,55.651156,55.209091,55.647724,55.498734,55.786302,55.351579,54.757704,...,55.662866,54.964789,55.212831,55.077696,57.691314,55.791996,55.982489,55.641598,55.222266,55.499530
2100~2400,86.116424,85.896434,84.783843,84.541284,85.460849,85.675906,85.015362,85.010403,83.810016,85.002454,...,83.269926,85.454610,87.396410,87.778173,90.311034,87.289864,85.295928,85.049183,85.470795,84.550902
2400~2700,87.551404,87.543651,88.557015,88.353504,87.743724,88.972823,88.556409,88.559566,89.371417,88.553348,...,88.194696,87.037244,86.586641,87.076319,91.261126,89.228636,88.082193,89.677976,89.232838,88.363401
2700~3000,57.046468,57.049040,57.183983,57.181393,56.608237,55.481664,56.464218,56.607912,56.319949,56.752666,...,56.885893,57.357475,57.809189,57.014671,60.487755,58.129656,57.710971,56.033476,56.373783,57.691882
3000~3300,69.878964,69.098777,70.095123,70.434806,70.435346,70.433402,70.634714,70.365408,70.431408,70.100046,...,70.721171,70.830863,70.561522,70.452023,71.297955,70.891555,70.431641,70.619874,70.238913,70.237866
3300~3600,90.549401,90.880902,89.807954,88.415812,89.411549,90.178872,88.217623,89.408589,90.011122,89.813792,...,87.702661,87.397658,87.238150,88.214669,90.525258,89.631202,90.066517,87.798031,89.424396,88.431149
3600~3900,63.765131,63.556501,63.549744,63.971871,62.920295,62.694399,64.613872,64.181685,62.269377,63.533013,...,63.999582,65.583776,67.619824,66.700405,70.420810,66.442472,64.335362,65.179403,63.231970,63.407963
3900~4200,95.435038,95.346884,95.522056,95.258801,95.522842,95.880626,95.523659,95.434343,95.868384,95.263003,...,96.053768,94.899378,94.456100,94.577464,97.059823,95.597712,94.659664,95.173778,95.356571,95.840592
4200~4500,70.327475,70.391903,70.455797,70.586481,70.521567,70.390679,70.455616,70.717911,70.716958,70.651575,...,70.942283,71.106810,70.729571,70.971843,71.784672,71.050845,70.951832,70.827824,70.761445,70.405045


STVM_35.xlsx 생성 완료


,-5,-4,-3,-2,-1,1,2,3,4,5,...,256,257,258,259,260,261,262,263,264,265
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,50.580736,50.650385,50.858706,50.787518,50.580713,50.785912,50.716212,50.850741,50.647371,50.314269,...,50.520542,51.107631,50.791189,50.676573,54.977657,50.999104,50.481369,50.651037,50.589189,50.526144
2100~2400,82.387381,82.087811,80.594331,80.288249,81.494655,81.787508,80.888015,80.881279,72.335780,80.870445,...,82.920790,80.923263,83.331633,81.196880,92.616533,84.414544,83.398463,81.552876,82.646343,81.225994
2400~2700,85.689666,85.679385,87.023266,86.753378,85.944713,87.567494,87.022463,87.026649,88.045143,87.018403,...,85.634531,87.462103,86.196822,86.547344,93.496753,90.841773,86.507356,86.553327,86.778975,86.764250
2700~3000,57.049725,57.052807,57.214514,57.211410,56.524577,55.174565,56.351993,56.524186,56.179100,56.697651,...,55.812118,55.613469,55.975126,58.672495,67.620005,57.170472,57.178872,56.624408,55.913799,56.860982
3000~3300,70.137195,69.595145,70.288725,70.695779,70.696426,70.694096,70.935335,70.612617,70.691707,70.294627,...,71.631521,71.441706,70.537405,68.526165,83.542499,71.328641,70.707032,70.616030,70.687531,70.842444
3300~3600,92.440891,92.771621,91.587614,90.174705,91.185296,91.996873,89.968782,91.182292,91.795853,91.593539,...,89.448857,89.367293,91.435935,92.178712,95.944188,92.920006,91.298060,91.723748,92.618361,89.459737
3600~3900,68.257472,67.943066,67.932885,68.569029,67.010276,66.686995,69.536523,68.885218,66.078743,67.907670,...,70.529277,68.705338,70.261030,70.051655,72.425361,70.395205,69.662237,66.969673,67.731478,70.015377
3900~4200,97.230553,97.124916,97.334830,97.019362,97.335772,97.643832,97.336751,97.229721,97.635852,97.024387,...,95.775504,96.563235,96.410793,97.626257,100.000000,98.036483,97.985645,97.670877,97.347822,97.138149
4200~4500,80.376549,80.674850,80.970670,81.575729,81.275185,80.669183,80.969832,82.184242,82.179830,81.877111,...,83.141233,83.089664,82.035086,72.438309,92.431515,83.659648,72.414607,72.477497,72.457155,81.897886


STVM_36.xlsx 생성 완료
